In [1]:
# load model and scaler and make predictions on new data
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
#scaler = load(open('C:/Users/Kuzmins/Documents/Jupiter/VKR/scaler.pkl', 'rb'))

model = keras.models.load_model('C:/Users/Kuzmins/Documents/Jupiter/VKR/models/VKR_mlp_var1')
scaler = joblib.load('scaler.pkl')

In [3]:
#Ввод данных пользователем dfuser
#dfuser = dfp.copy(deep = True)
#dfuser = dfuser.reset_index()
#dfuser = dfuser.drop(labels=range(1, 936), axis=0)
#dfuser.drop(['index'], inplace=True, axis=1)

dfuser = pd.DataFrame({'Sootnoshenie_Matrix_napolnitel' : [0], 'Plotnost_kg/m3' : [0], 
            'Modul_Uprugosti_GPa' : [0], 'Kolichestvo_napolnitela_m%' : [0], 
            'Soderjanie_epoxidnih_grupp_%_2' : [0], 'Temp_vspishki_C_2' : [0], 
            'Poverhnostnaia_plotnost_g/m2' : [0], 'Modul_uprugosti_pri_rastiajenii_GPa' : [0], 
            'Prochnost_pri_rastiajenii_Mpa' : [0], 'Potreblenie_smoli_g/m2' : [0], 
            'Ugol_nashivki_grad' : [0], 'Shag_nashivki' : [0], 'Plotnost_nashivki' : [0]})

col = dfuser.columns

print('Введите параметры:') 
a = 0 
for i in col: 
    if a != 0 and a != 7 and a !=8: 
        print(col[a]) 
        y = input() 
        dfuser[col[a]].values[0] = float(y) 
    else: 
        dfuser[col[a]].values[0] = 0 
    a+=1
    if a == 13:
        break
    

Введите параметры:
Plotnost_kg/m3
1
Modul_Uprugosti_GPa
1
Kolichestvo_napolnitela_m%
1
Soderjanie_epoxidnih_grupp_%_2
1
Temp_vspishki_C_2
1
Poverhnostnaia_plotnost_g/m2
1
Potreblenie_smoli_g/m2
1
Ugol_nashivki_grad
1
Shag_nashivki
1
Plotnost_nashivki
1


In [4]:
dfuser

,Sootnoshenie_Matrix_napolnitel,Plotnost_kg/m3,Modul_Uprugosti_GPa,Kolichestvo_napolnitela_m%,Soderjanie_epoxidnih_grupp_%_2,Temp_vspishki_C_2,Poverhnostnaia_plotnost_g/m2,Modul_uprugosti_pri_rastiajenii_GPa,Prochnost_pri_rastiajenii_Mpa,Potreblenie_smoli_g/m2,Ugol_nashivki_grad,Shag_nashivki,Plotnost_nashivki
0,0,1,1,1,1,1,1,0,0,1,1,1,1


In [5]:
#Нормализация with MinMaxScaler

modified = scaler.transform(dfuser)
dfuser_modified = pd.DataFrame(modified, columns = col)
dfuser_modified.drop(['Sootnoshenie_Matrix_napolnitel', 'Modul_uprugosti_pri_rastiajenii_GPa', 'Prochnost_pri_rastiajenii_Mpa'], inplace=True, axis=1)
dfuser_modified

,Plotnost_kg/m3,Modul_Uprugosti_GPa,Kolichestvo_napolnitela_m%,Soderjanie_epoxidnih_grupp_%_2,Temp_vspishki_C_2,Poverhnostnaia_plotnost_g/m2,Potreblenie_smoli_g/m2,Ugol_nashivki_grad,Shag_nashivki,Plotnost_nashivki
0,-4.729681,-0.000872,-0.263122,-1.108354,-0.862989,0.000307,-0.21223,0.011111,0.070272,-0.447279


In [6]:
pred = model.predict(dfuser_modified)
    
pred_inversed = pd.DataFrame([])
pred_inversed.at[0,0] = float(pred[0][0])

for i in range(1, 13):
    p = 0
    pred_inversed.at[0,i] = float(p)
    pred_inversed.at[0,7] = float(pred[0][1])
    pred_inversed.at[0,8] = float(pred[0][2])
    
Y_trans = scaler.inverse_transform(pred_inversed)

In [7]:
def predict(dfuser):
    model = keras.models.load_model('C:/Users/Kuzmins/Documents/Jupiter/VKR/models/VKR_mlp_var1')
    scaler = joblib.load('scaler.pkl')
    modified = scaler.transform(dfuser)
            
    pred = model.predict(dfuser_modified)
    
    pred_inversed = pd.DataFrame([])
    pred_inversed.at[0,0] = float(pred[0][0])

    for i in range(1, 13):
        p = 0
        pred_inversed.at[0,i] = float(p)
    pred_inversed.at[0,7] = float(pred[0][1])
    pred_inversed.at[0,8] = float(pred[0][2])
    
    Y_trans = scaler.inverse_transform(pred_inversed)
    
    return Y_trans

In [8]:
#Возвращает предсказанное значание (в размерности до нормализации)
Y_trans = scaler.inverse_transform(pred_inversed)

print(Y_trans[0,0], Y_trans[0,7], Y_trans[0,8])

1.892658435981896 65.55333591 1250.3928019999996


In [9]:
print(model.summary())

Model: "sequential_100"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_253 (Dense)            (None, 128)               1408      
_________________________________________________________________
dense_254 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_255 (Dense)            (None, 16)                1040      
_________________________________________________________________
dense_256 (Dense)            (None, 3)                 51        
_________________________________________________________________
dropout_100 (Dropout)        (None, 3)                 0         
_________________________________________________________________
dense_257 (Dense)            (None, 3)                 12        
Total params: 10,767
Trainable params: 10,767
Non-trainable params: 0
________________________________________________